In [ ]:
import os
from dotenv import load_dotenv
import weaviate

load_dotenv()

weaviate_apikey = os.getenv("WEAVIATE_APIKEY")
client = weaviate.Client(
    "http://localhost:8080",
    auth_client_secret=weaviate.auth.AuthApiKey(weaviate_apikey),
)

# How many paragraphs we have in the database?
client.query.aggregate("passage").with_meta_count().do()

In [ ]:
# Ask a sample question
question = {"concepts": ["What is Antibacterial Activity?"]}

result = (
    client.query.get("Passage", ["text_content", "topic", "preprocessor_id"])
    .with_near_text(question)
    .with_limit(10)
    .with_additional(["vector", "distance", "id"])
    .do()
)

In [ ]:
# What are the related paragraphs?
for x in result["data"]["Get"]["Passage"]:
    print(x["text_content"], "\n", "-" * 180)


In [ ]:
raw_vec = result["data"]["Get"]["Passage"][0]["_additional"]["vector"]
raw_vec


## With pre-filtering

In [14]:
where_filter = {
    "path": ["topic"],
    "operator": "Equal",
    "valueText": "covid-19",
}

# where_filter = {
#     "path": ["preprocessor_id"],
#     "operator": "Equal",
#     "valueText": "haystack-v0.0.1",
# }


result = (
    client.query.get("Passage", ["text_content", "topic", "preprocessor_id"])
    .with_where(where_filter)
    .with_near_text(question)
    .with_limit(10)
    .with_additional(["vector", "distance", "id"])
    .do()
)